In [122]:
#input: city or airport

#return: airline

In [123]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import json

In [124]:
headers = {
    "X-RapidAPI-Key": "9062d689c2mshe1b9e4c756a6482p165546jsn847c6c6ea1cc",
    "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
    }


def get_codes():
    
    location = input('Insert location:')
    print()
    
    url = "https://aerodatabox.p.rapidapi.com/airports/search/term"
    querystring = {"q":location.lower(), "limit":"10"}

    response = requests.get(url, headers=headers, params=querystring)
    codes = response.json()['items'] if 'items' in response.json() else location

    return codes


def choose_code():
    
    codes = get_codes()
    
    while type(codes) is str or len(set([code['shortName'] for code in codes])) != 1:
        
        if not codes:
            print('Try another location. Unable to find code for:', codes, '\n')
        else:
            print('More than one location found. Insert one of the following airports:', [code['shortName'] for code in codes], '\n')
        
        codes = get_codes()
    
    return codes[0]['icao']


def flight_between(from_, to_):
    url = f"https://aerodatabox.p.rapidapi.com/airports/icao/{from_}/stats/routes/daily"
    response = requests.get(url, headers=headers)
    resp_json = response.json()

    results_list = []
    routes = resp_json['routes']
    for i in range(0, len(routes)):
        route = routes[i]
        if 'icao' in route['destination'] and route['destination']['icao'] == to_:
            results_list.extend(route['operators'])


   
    json_list = json.dumps(results_list)
    return pd.json_normalize(results_list).T

In [ ]:
print('[DEPARTURE]')
from_ = choose_code()
print('[DESTINATION]')
to_ = choose_code()

flight_df = flight_between(from_, to_)
flight_df

[DEPARTURE]


In [ ]:
#inputs: airport, departure and arrival date
#returns: delay

In [ ]:
import requests


location = input('Please enter the exact icao code:').upper()
from_when = input('Please enter the time in this format:\n \"YYYY-MM-DDThh:mm"')
to_when = input('Please enter the time in this format:\n \"YYYY-MM-DDThh:mm"')

url = f"https://aerodatabox.p.rapidapi.com/airports/icao/{location}/delays/{from_when}/{to_when}"

headers = {
    "X-RapidAPI-Key": "9062d689c2mshe1b9e4c756a6482p165546jsn847c6c6ea1cc",
    "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
}

response = requests.request("GET", url, headers=headers)
print(response)
print(response.text)

In [ ]:
pd.json_normalize(response.json())